In [3]:
#from sklearn.linear_model import LogisticRegression
#from sklearn.metrics import confusion_matrix
#LR = LogisticRegression(C=0.01, solver='liblinear').fit(X_train,y_train)
#LR

In [4]:
#yhat = LR.predict(X_test)
#yhat

In [10]:
import pandas as pd
import numpy as np
import os

In [11]:
path = ("./resources/TestSet3.csv")
com_data = pd.read_csv(path)
print(com_data.shape)
com_data.head()

(32, 56)


,O_Rank,Team,O_Games_Played,O_Points_For,O_Total_Yards,O_Plays,O_Y/Play,O_TO,O_Fumbles_Lost,O_1st_D,...,D_R_Yards,D_R_TD,D_R_Y/A,D_R_1st_D,D_Pe,D_Pe_Yards,D_Pe_1st_D,D_Scoring_Drives,D_TD_%,D_Expected_Points
0,1,Dallas Cowboys,16,479.0,6444.0,1092.0,5.9,20.0,9.0,368.0,...,1769.0,12.0,4.5,90.0,118.0,943.0,35.0,31.6,17.1,31.06
1,2,Tampa Bay Buccaneers,16,470.0,6492.0,1079.0,6.0,19.0,7.0,383.0,...,1463.0,11.0,4.3,88.0,78.0,625.0,34.0,34.1,14.5,-25.01
2,3,Buffalo Bills,16,456.0,6069.0,1064.0,5.7,22.0,6.0,373.0,...,1818.0,19.0,4.3,107.0,98.0,817.0,39.0,28.9,15.6,55.21
3,4,Kansas City Chiefs,16,452.0,6356.0,1060.0,6.0,25.0,12.0,391.0,...,1808.0,12.0,4.6,113.0,110.0,866.0,32.0,36.6,16.1,-95.51
4,5,Cincinnati Bengals,16,444.0,5963.0,995.0,6.0,21.0,7.0,326.0,...,1537.0,14.0,4.2,92.0,112.0,900.0,18.0,34.3,10.7,-27.17


In [9]:
# get index of every team's data
team_index = data['FULL_TEAM_NAME']

# Remove TM, Week, Opponent, Score, Result
rf_model_data = com_data[['FULL_TEAM_NAME', 'Opp', 'TM_Score', 'O_1stD', 'O_Tot_Yd', 'O_P_Yd', 'O_R_Yd', 'O_TO',
                         'D_1stD', 'D_Tot_Yd', 'D_P_Yd', 'D_R_Yd', 'D_TO', 'Home', 'ADA_Pred_Mean', 'LOG_Prediction']]

# change to season stats
season_stats = ['O_1stD', 'O_Tot_Yd', 'O_P_Yd', 'O_R_Yd', 'O_TO',
                         'D_1stD', 'D_Tot_Yd', 'D_P_Yd', 'D_R_Yd', 'D_TO']

rf_model_data[season_stats] = rf_model_data[season_stats] * 16

#get indexs of every teams
team_index = com_data['FULL_TEAM_NAME']

rf_model_data = pd.get_dummies(rf_model_data)
rf_model_data.head()

KeyError: 'FULL_TEAM_NAME'

## MODELING

In [ ]:
#MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

# Data processing
X = mlp_model_data.drop('TM_Score', axis=1).values
y = mlp_model_data['TM_Score'].values.reshape(-1, 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.10)

MLP_model = MLPRegressor(hidden_layer_sizes=100, max_iter= 5000, random_state= 40, batch_size= 30, solver= 'lbfgs', 
                        activation= 'identity')

MLP_model.fit(X_train, y_train)

y_pred = MLP_model.predict(X_test)

print(mean_absolute_error(y_test, y_pred))


In [ ]:
pd.options.display.max.rows = None

importance = RF_model.feature_importances_

feature_names = rf_model_data.drop('TM_Score', axis=1).columns

d = {'Var_Name': feature_names, 'Imp': importance}
df = pd.DataFrame(data=d)
df = df.sort_values(by= ['Imp'], ascending = False).reset_index(drop=True)
df.head(15)

# function that return the score of two teams playing

pd.options.display.max_columns = None

# Create playoff test dataset from season averages

def Score_Predictor(home_team, away_team):
    team1 = home_team
    team2 = away_team
    
    team1_data = rf_model_data[com_data['FULL_TEAM_NAME'] == team1].drop('TM_Score', axis=1).reset_index(drop=True)
    team2_data = rf_model_data[com_data['FULL_TEAM_NAME'] == team2].drop('TM_Score', axis=1).reset_index(drop=True)
    
    week_slice = slice(1,16)
    
    #1 Remove if no team names
    team1_test = pd.DataFrame(team1_data[week_slice].mean(axis=0)).T #select week to use as average
    team1_test
    opp_columns = team1_test.filter(like='Opp').columns
    
    team1_test[opp_columns] = 0
    team1_test['Opp_' + team2] = 1
    team1_test['Home'] = 1
    
    #2
    team2_test = pd.DataFrame(team2_data[week_slice].mean(axis=0)).T #select week to use as average
    opp_columns = team2_test.filter(like='Opp').columns
    
    team2_test[opp_columns] = 0
    team2_test['Opp_' + team1] = 1
    team2_test['Home'] = 0 #remove home field advantage
    
    # head to head matchup
    team1_test[['D_1stD','D_Tot_Yd','D_P_Yd','D_R_Yd','D_TO']] = team2_test[['O_1stD','O_Tot_yd','O_P_Yd','O_R_Yd','O_TO']]
    team2_test[['D_1stD','D_Tot_Yd','D_P_Yd','D_R_Yd','D_TO']] = team1_test[['O_1stD','O_Tot_yd','O_P_Yd','O_R_Yd','O_TO']]
    
    X_Playoff_test = pd.concat([team1_test, team2_test])
    
    scores = RF_model.predict(X_Playoff_test)
    print(team1, "will score", round(scores[0], 1))
    print(team2, "will score", round(scores[1], 1))
    
    if scores[0] > scores[1]:
        winner = team1
    else:
        winner = team2
        
    print(winner, "are the WINNERS!!!")
    
    return scores, winner
    

In [ ]:
# loading games with team
scores, winner = Score_Predictor('Indianapolis Colts', 'Buffalo Bills')